# Testing sim scripts.

In [ ]:
from shared.preface import *

box_name = 'L025N752'
box_ver = 'DMONLY/SigmaConstant00'
out_dir = f'{box_name}/{box_ver}'
nums_snaps = np.load(f'{out_dir}/nums_snaps.npy')
zeds = np.load(f'{out_dir}/zeds_snaps.npy')
zint = np.load(f'{out_dir}/z_int_steps.npy')
sint = np.load(f'{out_dir}/s_int_steps.npy')
print(nums_snaps[:10])
print(zeds[:10])
print(zint[:10])
print(sint[:10])

# from shared.functions import read_DM_halo_index

# with h5py.File(f'{out_dir}/MergerTree.hdf5') as tree:
#     prog_IDs = tree['Assembly_history/Progenitor_index'][28,:]
#     prog_IDs_np = np.array(np.expand_dims(prog_IDs, axis=1), dtype=int)

# with open(f'{out_dir}/box_parameters.yaml', 'r') as file:
#     box_setup = yaml.safe_load(file)

# box_file_dir = box_setup['File Paths']['Box File Directory']
# print(box_file_dir, type(box_file_dir))
# print(prog_IDs_np.flatten())

# np.savetxt('prog_IDs.txt', prog_IDs_np.flatten(), fmt='%d')

idx_arr = np.zeros(len(zint))
for i, s_val in enumerate(sint):
    z = np.interp(s_val, sint, zint)
    idx_arr[i] = np.abs(zeds - z).argmin()
print(idx_arr[::10])

snaps_GRID_L = np.load(f'{out_dir}/snaps_GRID_L.npy')/kpc
snaps_DM_num = np.load(f'{out_dir}/snaps_DM_num.npy')
snaps_DM_com = np.load(f'{out_dir}/snaps_DM_com.npy')/kpc
snaps_QJ_abs = np.load(f'{out_dir}/snaps_QJ_abs.npy')

# plt.plot(zint, np.take(snaps_DM_num, np.asarray(idx_arr, dtype=int))); plt.show()
plt.plot(zeds[::-1], snaps_DM_num); plt.show()

In [ ]:
# !python simulation_analytical.py -d L025N752/DMONLY/SigmaConstant00 --MW_halo --no-VC_halo --no-AG_halo

In [ ]:
# out_dir = f'L025N752/DMONLY/SigmaConstant00'
# output = np.load(f'{out_dir}/neutrino_vectors_analytical_batch1.npy')
# print(output.shape)

In [ ]:
!python simulation_numerical.py -d L025N752/DMONLY/SigmaConstant00 -st single_halos -mg 12.0 -mr 0.6 -hn 1

# Final Content of notebook.

## Load, transform and plot simulation outputs.

In [ ]:
from shared.preface import *
from shared.shared_functions import velocity_to_momentum

def transform_simulation_outputs(out_dir, sim_fullname):
    """
    Input sim_fullname specifies the simulation mode with additional information: e.g. "single_halos_40k_test1" or "spheres_5shells_10k_final".
    """

    # Try loading the equivalent output for the analytical simulation method.
    # If it hasn't been run yet, raise error and inform user.
    try:

        # Load neutrino vectors (positions and velocities) of the 10k batches, and concatenate them into a single array.
        batch_paths = glob.glob(
            f'{out_dir}/neutrino_vectors_analytical_batch*.npy'
        )
        vectors_ana = []
        for batch_path in batch_paths:
            vectors_ana.append(np.load(batch_path))
        vectors_ana = np.squeeze(np.array(vectors_ana))

        etas_ana = np.load(
            f'{out_dir}/number_densities_analytical.npy'
        )/N0

        analytical_out = True

    except FileNotFoundError:
        print('! Analytical simulation output not found !')
        analytical_out = False


    if 'single_halos' or 'spheres' in sim_fullname:
        
        # Load neutrino vectors (positions and velocities) of the 10k batches, and concatenate them into a single array.
        batch_paths = glob.glob(
            f'{out_dir}/neutrino_vectors_numerical_batch*.npy'
        )
        vectors_num = []
        for batch_path in batch_paths:
            vectors_num.append(np.load(batch_path))
        vectors_num = np.squeeze(np.array(vectors_num))

        etas_num = np.load(
            f'{out_dir}/number_densities_numerical.npy'
        )/N0

        if analytical_out:
            return vectors_ana, etas_ana, vectors_num, etas_num
        else:
            return vectors_num, etas_num
    
    else:
        # Load angle pairs and calculate overdensities for the all_sky mode.
        all_sky_output = np.load(f'{out_dir}/number_densities_numerical.npy')
        angle_pairs = all_sky_output[:, :2]
        etas_numerical = all_sky_output[:, 2:]/N0

        return etas_numerical, angle_pairs


def analyze_simulation_outputs(
        etas, m_nu_range,
        box_name, box_ver,
        plots_to_make, ylims, Mertsch=False,
        # vectors, sim_fullname,
    ):

    # Figure directory.
    # fig_dir = f'figures/{box_name}/{box_ver}/{sim_fullname}'
    fig_dir = f'figures/{box_name}/{box_ver}'
    if not os.path.exists(fig_dir):
        os.makedirs(fig_dir)

    savefig_args = dict(
        bbox_inches='tight'
    )

    if Mertsch:
        etas_analytical = etas[0,:]
        etas_numerical = etas[1:,:]
    else:
        etas_numerical = etas[...]

    ### ================================== ###
    ### Figure of merit: overdensity band. ###
    ### ================================== ###

    fig, ax = plt.subplots(1,1)

    if 'overdensity_band' in plots_to_make:

        if Mertsch:
            # Plot smooth simulation.
            ax.plot(
                m_nu_range*1e3, (etas_analytical-1), color='red', ls='solid', 
                label='Analytical simulation'
            )

        # '''
        if etas_numerical.ndim <= 1:
            ax.plot(
                m_nu_range*1e3, (etas_numerical-1), color='blue', 
                label='medians'
            )
        else:
            nus_median = np.median(etas_numerical, axis=0)
            nus_perc2p5 = np.percentile(etas_numerical, q=2.5, axis=0)
            nus_perc97p5 = np.percentile(etas_numerical, q=97.5, axis=0)
            nus_perc16 = np.percentile(etas_numerical, q=16, axis=0)
            nus_perc84 = np.percentile(etas_numerical, q=84, axis=0)
            ax.plot(
                m_nu_range*1e3, (nus_median-1), color='blue', 
                label='Halo sample medians'
            )
            ax.fill_between(
                m_nu_range*1e3, (nus_perc2p5-1), (nus_perc97p5-1), 
                color='blue', alpha=0.2, label='2.5-97.5 %'
            )
            ax.fill_between(
                m_nu_range*1e3, (nus_perc16-1), (nus_perc84-1), 
                color='blue', alpha=0.3, label='16-84 %'
            )
        # '''

        if Mertsch:
            # Plot endpoint values from Mertsch et al.
            x_ends = [1e1, 3*1e2]
            y_ends = [3*1e-3, 4]
            ax.scatter(x_ends, y_ends, marker='x', s=15, color='orange')

        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_title(f'Overdensity band')
        ax.set_xlabel(r'$m_{\nu}$ [meV]')
        ax.set_ylabel(r'$n_{\nu} / n_{\nu, 0}$')
        # ax.set_ylim(ylims[0], ylims[1])
        plt.grid(True, which="both", ls="-")
        plt.legend(loc='lower right')


        plt.savefig(fig_dir, **savefig_args)
        plt.show()

    ### ================================== ###
    ### Figure of merit: overdensity band. ###
    ### ================================== ###


def number_density(p0, p1, pix_sr=4*Pi):
    """Neutrino number density obtained by integration over initial momenta.

    Args:
        p0 (array): neutrino momentum today
        p1 (array): neutrino momentum at z_back (final redshift in sim.)

    Returns:
        array: Value of relic neutrino number density in (1/cm^3).
    """    

    g = 2.  # 2 degrees of freedom per flavour: particle and anti-particle
    
    # note: trapz integral method needs sorted (ascending) "x-axis" array.
    ind = p0.argsort(axis=-1)
    p0_sort = np.take_along_axis(p0, ind, axis=-1)
    p1_sort = np.take_along_axis(p1, ind, axis=-1)

    # Fermi-Dirac value with momentum at end of sim.
    # Function expit from scipy equivalent to 1/(np.exp(-X)+1).
    # (thus the minus sign)
    FDvals = expit(-p1_sort/T_CNB)

    # Calculate number density.
    y = p0_sort**3 * FDvals
    x = p0_sort
    n_raw = np.trapz(y, np.log(x), axis=-1)

    # Multiply by constants and/or solid angles and convert to 1/cm**3.
    n_cm3 = pix_sr * g/((2*Pi)**3) * n_raw / (1/cm**3)

    return n_cm3


def number_densities_mass_range(
    sim_vels, nu_masses, out_file, pix_sr=4*Pi, sim_type='single_halos'
):
    
    # Convert velocities to momenta.
    p_arr, _ = velocity_to_momentum(sim_vels, nu_masses)

    # if average:
    #     inds = np.array(np.where(FCT_zeds >= z_start)).flatten()
    #     temp = [
    #         number_density(p_arr[...,0], p_arr[...,k], pix_sr) for k in inds
    #     ]
    #     num_densities = np.mean(np.array(temp.T), axis=-1)
    # else:
    num_densities = number_density(p_arr[...,0], p_arr[...,-1], pix_sr)

    if sim_type == 'all_sky':
        return num_densities
    else:
        np.save(f'{out_file}', num_densities)



### -------------------- ###
### Automatic procedure. ###
### -------------------- ###
box_name = 'L025N752'
box_ver = 'DMONLY/SigmaConstant00'
out_dir = f'{box_name}/{box_ver}'
vecs_ana, etas_ana, vecs_num, etas_num = transform_simulation_outputs(
    out_dir, sim_fullname='single_halos'
)
# etas_combined = np.stack((etas_ana, etas_num), axis=0)
etas_combined = etas_num

neutrino_massrange = np.load(f'{out_dir}/neutrino_massrange_eV.npy')*eV

### ----------------- ###
### Manual procedure. ###
### ----------------- ###
'''
fname_ana = f'{out_dir}/neutrino_densities_analytical_log.npy'
number_densities_mass_range(
    sim_vels=vecs_ana[...,3:6],
    nu_masses=neutrino_massrange,
    out_file=fname_ana,
    sim_type='single_halos'
)
fname_num = f'{out_dir}/neutrino_densities_numerical_log.npy'
number_densities_mass_range(
    sim_vels=vecs_num[...,3:6],
    nu_masses=neutrino_massrange,
    out_file=fname_num,
    sim_type='single_halos'
)
etas_ana_log = np.load(fname_ana)/N0
etas_num_log = np.load(fname_num)/N0
etas_combined = np.stack((etas_ana_log, etas_num_log), axis=0)
'''


analyze_simulation_outputs(
    etas=etas_combined, 
    m_nu_range=neutrino_massrange,
    box_name=box_name, 
    box_ver=box_ver,
    plots_to_make=('overdensity_band', 'vels'), 
    ylims=(1e-3, 1e2),
    # ylims=(0, 1e-2),
    Mertsch=False
)